Ideal student-teacher ratio across Texas districts

In [20]:
## Project 4
import pandas as pd
import sklearn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from pymongo import MongoClient

In [21]:
mongo = MongoClient(port=27017)

In [22]:
for dbname in mongo.list_database_names():
    if dbname == "texasSchoolsDB":
        print(f'{dbname} <----------')
    else:
        print(dbname)

admin
classDB
config
dbName
fruits_db
local
met
texasSchoolsDB <----------
travel_db
uk_food


In [23]:
db = mongo["texasSchoolsDB"]

In [24]:

collection = db['teachers20_21']

In [25]:
teacher20_21_df = pd.DataFrame(list(collection.find()))

teacher20_21_df.drop(columns=['_id'], inplace=True, errors='ignore')

teacher20_21_df.head()

,DISTRICT NAME,TOTAL NUMBER OF SCHOOLS,TOTAL STUDENTS,TOTAL STAFF FTE,TOTAL TEACHER FTE,STAFF: % TEACHERS,STAFF: % EDUCATIONAL AIDES,STAFF: % AUXILIARY,AVERAGE SALARY: TEACHER,NUMBER OF STUDENTS PER TOTAL STAFF,...,TEACHER: % CAREER & TECHNICAL EDUCATION,TEACHER: % OTHER EDUCATION (INCLUDES G & T),ATTENDANCE RATE (2020-21),ANNUAL DROPOUT RATE GR,4-YR LONGITUDINAL GRADUATION RATE (CLASS OF 2021) DISTRICT EXCL,5-YR LONGITUDINAL GRADUATION RATE (CLASS OF 2020) DISTRICT EXCL,ANNUAL GRADUATE COUNT (2020-21),COLLEGE ADMISSIONS: % AT/ABOVE CRITERION (2020-2021),SAT-AVERAGE SCORE (2021-2020),ACT-AVERAGE SCORE (2021-2020)
0,CAYUGA ISD,3,574,99.9,46.7,46.8,14.8,26.5,55570.0,5.7,...,8.5,0.0,96.7,{' 9-12 (2020-21)': 0},100.0,100.0,36.0,19.0,980.0,NaN
1,ELKHART ISD,4,1150,186.6,104.9,56.2,16.2,19.5,47916.0,6.2,...,13.0,1.5,96.0,{' 9-12 (2020-21)': 0.3},100.0,98.9,91.0,27.7,979.0,-1.0
2,FRANKSTON ISD,3,808,146.7,74.5,50.8,15.0,27.4,50382.0,5.5,...,2.2,0.0,95.4,{' 9-12 (2020-21)': 0.4},95.2,100.0,41.0,36.8,980.0,-1.0
3,NECHES ISD,2,342,60.1,30.2,50.3,13.7,26.2,55346.0,5.7,...,10.8,1.7,95.8,{' 9-12 (2020-21)': 0},95.8,97.0,23.0,15.0,1007.0,18.8
4,PALESTINE ISD,6,3360,553.4,260.3,47.0,19.7,19.5,48825.0,6.1,...,9.0,1.8,93.7,{' 9-12 (2020-21)': 0},99.0,99.6,201.0,49.4,1048.0,21.0


Project Overview
* Goal: Develop a predictive model for the ideal student-teacher ratio.
* Data: District-wise information from Texas.
* Metrics: To evaluate our model, we'll consider accuracy measures like MAE (Mean Absolute Error), RMSE (Root Mean Square Error), and R² (Coefficient of Determination).

In [26]:
# Check for missing values
print(teacher20_21_df.isnull().sum())

DISTRICT NAME                                                        0
TOTAL NUMBER OF SCHOOLS                                              0
TOTAL STUDENTS                                                       0
TOTAL STAFF FTE                                                      3
TOTAL TEACHER FTE                                                    3
STAFF: % TEACHERS                                                    3
STAFF: % EDUCATIONAL AIDES                                           3
STAFF: % AUXILIARY                                                   3
AVERAGE SALARY: TEACHER                                              4
NUMBER OF STUDENTS PER TOTAL STAFF                                   3
NUMBER OF STUDENTS PER TEACHER                                       3
TEACHER: % WITH 5 OR FEWER YEARS OF EXPERIENCE                       3
TEACHER: AVERAGE YEARS OF EXPERIENCE                                 3
TEACHER: % WITH ADVANCED DEGREES                                     3
TEACHE

In [27]:
from sklearn.impute import SimpleImputer

# Imputer for numeric columns
numeric_imputer = SimpleImputer(strategy='mean')
# List of numeric columns with minimal missing values
numeric_columns_minimal_missing = ['TOTAL STAFF FTE', 'TOTAL TEACHER FTE', 'AVERAGE SALARY: TEACHER']

teacher20_21_df[numeric_columns_minimal_missing] = numeric_imputer.fit_transform(teacher20_21_df[numeric_columns_minimal_missing])

# If any of the columns with minimal missing values are categorical, use mode imputation similarly.


In [18]:
# Example: Imputation for a column with many missing values
# Let's use median for these to avoid the influence of outliers
substantial_missing_imputer = SimpleImputer(strategy='median')
teacher20_21_df[['COLLEGE ADMISSIONS: % AT/ABOVE CRITERION (2020-2021)', 'SAT-AVERAGE SCORE (2021-2020)', 'ACT-AVERAGE SCORE (2021-2020)']] = substantial_missing_imputer.fit_transform(teacher20_21_df[['COLLEGE ADMISSIONS: % AT/ABOVE CRITERION (2020-2021)', 'SAT-AVERAGE SCORE (2021-2020)', 'ACT-AVERAGE SCORE (2021-2020)']])


In [31]:
for column in ['COLLEGE ADMISSIONS: % AT/ABOVE CRITERION (2020-2021)', 'SAT-AVERAGE SCORE (2021-2020)', 'ACT-AVERAGE SCORE (2021-2020)']:
    teacher20_21_df[column + '_missing'] = teacher20_21_df[column].isnull().astype(int)

In [30]:
teacher20_21_df.columns

Index(['DISTRICT NAME', 'TOTAL NUMBER OF SCHOOLS', 'TOTAL STUDENTS',
       'TOTAL STAFF FTE', 'TOTAL TEACHER FTE', 'STAFF: % TEACHERS',
       'STAFF: % EDUCATIONAL AIDES', 'STAFF: % AUXILIARY',
       'AVERAGE SALARY: TEACHER', 'NUMBER OF STUDENTS PER TOTAL STAFF',
       'NUMBER OF STUDENTS PER TEACHER',
       'TEACHER: % WITH 5 OR FEWER YEARS OF EXPERIENCE',
       'TEACHER: AVERAGE YEARS OF EXPERIENCE',
       'TEACHER: % WITH ADVANCED DEGREES', 'TEACHER: % AFRICAN AMERICAN',
       'TEACHER: % HISPANIC', 'TEACHER: % WHITE', 'TEACHER: % AMERICAN INDIAN',
       'TEACHER: % ASIAN', 'TEACHER: % PACIFIC ISLANDER',
       'TEACHER: % TWO OR MORE RACES', 'TEACHER: % REGULAR EDUCATION',
       'TEACHER: % SPECIAL EDUCATION', 'TEACHER: % COMPENSATORY EDUCATION',
       'TEACHER: % BILINGUAL/ESL EDUCATION',
       'TEACHER: % CAREER & TECHNICAL EDUCATION',
       'TEACHER: % OTHER EDUCATION (INCLUDES G & T)',
       'ATTENDANCE RATE (2020-21)', 'ANNUAL DROPOUT RATE GR',
       '4-YR LONG

Idea> 1. Proportion of Teachers with Advanced Degrees
A higher percentage of teachers with advanced degrees might correlate with a different optimal student-teacher ratio, potentially reflecting on the quality of education.

Idea> 2. Experience Level
The average years of experience among teachers and the percentage of teachers with 5 or fewer years of experience could be crucial. These could affect the ideal ratio, as more experienced teachers might manage larger classes more effectively.

Idea> 3. 3. Teacher Diversity
Diversity in the teaching staff, represented by the percentage of teachers from various racial backgrounds, could be an important factor, possibly affecting student engagement and the learning environment.

Idea> 4. Specialization in Education
The percentage of teachers in specialized roles (Special Education, Bilingual/ESL Education, etc.) might influence the ideal student-teacher ratio, as these roles often require more individualized attention to students.

Idea> 5. 5. Financial Aspects
The average salary of a teacher could be a proxy for district investment in education, which might relate to the optimal student-teacher ratio.

In [32]:

# Creating column current student/teacher ratio
if 'TOTAL STUDENTS' in teacher20_21_df.columns and 'TOTAL TEACHER FTE' in teacher20_21_df.columns:
    # Calculate the current student-teacher ratio
    teacher20_21_df['current_student_teacher_ratio'] = teacher20_21_df['TOTAL STUDENTS'] / teacher20_21_df['TOTAL TEACHER FTE']
    print("The 'current_student_teacher_ratio' feature has been successfully added to the dataset.")
else:
    print("Error: Required columns ('TOTAL STUDENTS' and/or 'TOTAL TEACHER FTE') are missing from the dataset.")

# Display the first few rows to verify the new column
print(teacher20_21_df[['TOTAL STUDENTS', 'TOTAL TEACHER FTE', 'current_student_teacher_ratio']].head())


The 'current_student_teacher_ratio' feature has been successfully added to the dataset.
   TOTAL STUDENTS  TOTAL TEACHER FTE  current_student_teacher_ratio
0             574               46.7                      12.291221
1            1150              104.9                      10.962822
2             808               74.5                      10.845638
3             342               30.2                      11.324503
4            3360              260.3                      12.908183


In [33]:
teacher20_21_df.columns

Index(['DISTRICT NAME', 'TOTAL NUMBER OF SCHOOLS', 'TOTAL STUDENTS',
       'TOTAL STAFF FTE', 'TOTAL TEACHER FTE', 'STAFF: % TEACHERS',
       'STAFF: % EDUCATIONAL AIDES', 'STAFF: % AUXILIARY',
       'AVERAGE SALARY: TEACHER', 'NUMBER OF STUDENTS PER TOTAL STAFF',
       'NUMBER OF STUDENTS PER TEACHER',
       'TEACHER: % WITH 5 OR FEWER YEARS OF EXPERIENCE',
       'TEACHER: AVERAGE YEARS OF EXPERIENCE',
       'TEACHER: % WITH ADVANCED DEGREES', 'TEACHER: % AFRICAN AMERICAN',
       'TEACHER: % HISPANIC', 'TEACHER: % WHITE', 'TEACHER: % AMERICAN INDIAN',
       'TEACHER: % ASIAN', 'TEACHER: % PACIFIC ISLANDER',
       'TEACHER: % TWO OR MORE RACES', 'TEACHER: % REGULAR EDUCATION',
       'TEACHER: % SPECIAL EDUCATION', 'TEACHER: % COMPENSATORY EDUCATION',
       'TEACHER: % BILINGUAL/ESL EDUCATION',
       'TEACHER: % CAREER & TECHNICAL EDUCATION',
       'TEACHER: % OTHER EDUCATION (INCLUDES G & T)',
       'ATTENDANCE RATE (2020-21)', 'ANNUAL DROPOUT RATE GR',
       '4-YR LONG

For starters, we'll explore a few regression models to see which performs best on the dataset:

Linear Regression: A good baseline that's easy to interpret.
Random Forest Regressor: A powerful model that can capture complex nonlinear relationships.
Gradient Boosting Regressor: Another robust model known for its predictive accuracy.

X = 'TOTAL NUMBER OF SCHOOLS': More schools could indicate a larger district with potentially different staffing needs.

'TOTAL STUDENTS': Directly impacts the student-teacher ratio.

'TOTAL TEACHER FTE': Essential for calculating the student-teacher ratio.

'STAFF: % TEACHERS', 'STAFF: % EDUCATIONAL AIDES', and 'STAFF: % AUXILIARY': These percentages provide insight into the composition of the staff, which could affect the ideal ratio.

'AVERAGE SALARY: TEACHER': Higher salaries might attract more qualified or experienced teachers, potentially affecting the ideal ratio.

'TEACHER: % WITH ADVANCED DEGREES' and 'TEACHER: AVERAGE YEARS OF EXPERIENCE': Qualifications and experience of teachers could influence the optimal student-teacher ratio.

'TEACHER: % WITH 5 OR FEWER YEARS OF EXPERIENCE': This might indicate a younger, less experienced workforce, possibly affecting the ideal ratio differently.

Demographic percentages ('TEACHER: % AFRICAN AMERICAN', 'TEACHER: % HISPANIC', etc.): While not directly affecting the student-teacher ratio, these features might be relevant in broader analyses related to diversity and educational equity.


In [35]:


# Check if any division might lead to infinity
teacher20_21_df['current_student_teacher_ratio'] = teacher20_21_df['TOTAL STUDENTS'] / teacher20_21_df['TOTAL TEACHER FTE'].replace(0, np.nan)

# For 'normalized_teacher_salary', ensure the normalization process didn't introduce any infinity
# Assuming 'normalized_teacher_salary' is already in your DataFrame, no division by zero should occur in normalization
# However, let's ensure no infinite values are present
teacher20_21_df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Now, handle NaN values (which might include what were infinities)
teacher20_21_df.dropna(subset=['current_student_teacher_ratio'], inplace=True)


Finding best column to include in X

In [37]:
# Select only numeric columns for correlation analysis
numeric_data = teacher20_21_df.select_dtypes(include=['number'])

# Now, calculate the correlation matrix on this numeric subset
correlation_matrix = numeric_data.corr()

# Extract and sort the correlations with 'current_student_teacher_ratio'
correlation_with_target = correlation_matrix['current_student_teacher_ratio'].sort_values(ascending=False)

print("Correlation with 'current_student_teacher_ratio':")
print(correlation_with_target)



Correlation with 'current_student_teacher_ratio':
current_student_teacher_ratio                                      1.000000
NUMBER OF STUDENTS PER TOTAL STAFF                                 0.827062
NUMBER OF STUDENTS PER TEACHER                                     0.369427
TEACHER: % OTHER EDUCATION (INCLUDES G & T)                        0.276473
TEACHER: % WITH ADVANCED DEGREES                                   0.244428
AVERAGE SALARY: TEACHER                                            0.197899
COLLEGE ADMISSIONS: % AT/ABOVE CRITERION (2020-2021)               0.174432
SAT-AVERAGE SCORE (2021-2020)                                      0.170024
ANNUAL GRADUATE COUNT (2020-21)                                    0.154119
TOTAL STUDENTS                                                     0.150785
TEACHER: % BILINGUAL/ESL EDUCATION                                 0.135184
TOTAL NUMBER OF SCHOOLS                                            0.134044
TOTAL STAFF FTE                       

Defining a better X value

In [38]:
# Define the feature set for X based on the correlation analysis and domain relevance
selected_features = [
    'NUMBER OF STUDENTS PER TOTAL STAFF',  # Strong positive correlation
    'TEACHER: % OTHER EDUCATION (INCLUDES G & T)',  # Indicates specialized education focus
    'TEACHER: % WITH ADVANCED DEGREES',  # Quality of education staff
    'AVERAGE SALARY: TEACHER',  # Reflects district investment and possibly cost-of-living
    'TOTAL STUDENTS',  # Basic metric of district size
    'TOTAL NUMBER OF SCHOOLS',  # Reflects the scope of the district
    # Demographic and staffing composition
    'TEACHER: % AFRICAN AMERICAN',
    'TEACHER: % HISPANIC',
    'STAFF: % TEACHERS',  # Composition of staff
    'STAFF: % EDUCATIONAL AIDES',
    'STAFF: % AUXILIARY'
]

# Assuming 'teachers20_21' is your DataFrame
X = teacher20_21_df[selected_features]

# Ensure X does not contain any missing values
# You can decide to fill missing values with the median or mean, or drop rows/columns with missing values
X.fillna(X.median(), inplace=True)  # This is just one way to handle missing values

# Print the first few rows of X to verify
print(X.head())


   NUMBER OF STUDENTS PER TOTAL STAFF  \
0                                 5.7   
1                                 6.2   
2                                 5.5   
3                                 5.7   
4                                 6.1   

   TEACHER: % OTHER EDUCATION (INCLUDES G & T)  \
0                                          0.0   
1                                          1.5   
2                                          0.0   
3                                          1.7   
4                                          1.8   

   TEACHER: % WITH ADVANCED DEGREES  AVERAGE SALARY: TEACHER  TOTAL STUDENTS  \
0                              14.8                  55570.0             574   
1                              19.0                  47916.0            1150   
2                              30.7                  50382.0             808   
3                               9.6                  55346.0             342   
4                              15.4                 

/var/folders/vf/_j2gkjzn02s0myp33_2rv0gr0000gn/T/ipykernel_54186/2105023841.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.fillna(X.median(), inplace=True)  # This is just one way to handle missing values


In [39]:
y = teacher20_21_df['current_student_teacher_ratio']

In [40]:
#Try a simple Linear Regression Model with new X (same y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [41]:
# Initialize the Linear Regression model
model = LinearRegression()

# Fit the model on the training data
model.fit(X_train, y_train)

# Predict on the testing set
y_pred = model.predict(X_test)


In [42]:
# Calculate the model's performance metrics
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Linear Regression R² score: {r2:.4f}")
print(f"Linear Regression Mean Absolute Error (MAE): {mae:.4f}")
print(f"Linear Regression Root Mean Square Error (RMSE): {rmse:.4f}")


Linear Regression R² score: 0.8403
Linear Regression Mean Absolute Error (MAE): 0.8951
Linear Regression Root Mean Square Error (RMSE): 1.4360


With a Linear Regression R² score of 0.8423, your model explains a substantial proportion of the variance in the current_student_teacher_ratio, which is quite good for many practical purposes. The Mean Absolute Error (MAE) of 0.9674 and the Root Mean Square Error (RMSE) of 1.4794 further suggest that, on average, your model's predictions are within approximately one unit of the actual student-teacher ratios. These are encouraging results, indicating that the model, with the selected features, is performing well in predicting the student-teacher ratio.

R² Score (0.8423): Indicates a strong positive relationship between the features you selected and the current_student_teacher_ratio. This score suggests that 84.23% of the variance in the student-teacher ratio can be explained by the model, which is considered high in many contexts.

MAE (0.9674): This value tells us that, on average, the predictions of the student-teacher ratio by the model are less than one unit away from the actual values. This level of error may be acceptable, depending on the context and how the predictions are to be used.

RMSE (1.4794): Since RMSE is more sensitive to larger errors than MAE, this value suggests that there aren't many large errors in the predictions, which is good. However, it's slightly higher than the MAE, indicating some predictions were notably off.

In [43]:
import statsmodels.api as sm

# Assuming 'X' and 'y' are already defined
# Add a constant to the predictor variable set to represent the intercept
X_with_const = sm.add_constant(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_with_const, y, test_size=0.2, random_state=42)


In [45]:
# Fit the regression model using the training set
model = sm.OLS(y_train, X_train).fit()

# Print out the statistics
model_summary = model.summary()
print(model_summary)



                                  OLS Regression Results                                 
Dep. Variable:     current_student_teacher_ratio   R-squared:                       0.872
Model:                                       OLS   Adj. R-squared:                  0.871
Method:                            Least Squares   F-statistic:                     589.5
Date:                           Mon, 08 Apr 2024   Prob (F-statistic):               0.00
Time:                                   22:42:18   Log-Likelihood:                -2024.8
No. Observations:                            964   AIC:                             4074.
Df Residuals:                                952   BIC:                             4132.
Df Model:                                     11                                         
Covariance Type:                       nonrobust                                         
                                                  coef    std err          t      P>|t|      [0.025 

In [46]:
# Predicting with the test set
y_pred = model.predict(X_test)


In [47]:
from sklearn.metrics import mean_squared_error
import numpy as np

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'RMSE: {rmse:.4f}')


RMSE: 1.4360
